In [1]:
import os, csv, random, time

from __future__ import division
from matplotlib import  pyplot as plt
from scipy import stats, optimize
from collections import deque
from statsmodels.base.model import GenericLikelihoodModel
from scipy.stats import bernoulli,poisson,norm,expon
# from pathos.multiprocessing import ProcessingPool as Pool

import numpy as np
import scipy.io as sio
import seaborn as sns

%matplotlib inline

In [ ]:
class Pairwise_Newton(GenericLikelihoodModel):
    
    """
    Newton's method's last output is NaN for some reason, so we have to collect the second to last result,
    which is stored in self.rank[0]
    """
    
    def __init__(self, endog, video_num, video_score, exog=None, **kwds):
        super(Pairwise, self).__init__(endog, exog, **kwds)
        self.rank =  deque([[1],[1]])
        self.w_hat = []
        self.video_num = video_num
        self.pairs_truth = self.endog
        self.total_pairs = len(self.endog)
        self.v = []
        self.w_star = np.array(video_score)
        
     
    def test_data_generation_sparsity(self, showResult=False, Nihar=False, Isabelle=False):
        
        """
        Shows how the error evolves with sparser data
        """
        
        all_data = self.endog
        
        num_test_pairs = (np.array((1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1))* self.total_pairs).astype(int)
        
        output_error_nihar = []
        output_error_isabelle = []

        for test_pair_num in num_test_pairs:
            
            print 'Current evaluating with %d test pairs' % test_pair_num
            
            test_pairs = [self.pairs_truth[i] for i in random.sample(range(self.total_pairs), test_pair_num)]
            self.endog = np.array(test_pairs)
            
            w_init = np.random.uniform(-5,5,video_num)
            self.fit(w_init)
            
            if showResult:
                res = self.rank[0]
                compare_rank(self.w_star, res, False, True)
                plt.legend([str(i) for i in num_test_pairs])  
            if Nihar:
                output_error_nihar.append(self.performance_nihar())
            if Isabelle:
                output_error_isabelle.append(self.performance_isabelle())
                
        self.endog = all_data
        return output_error_nihar, output_error_isabelle
        
        
    def test_data_generation_noise(self, showResult=False, Nihar=False, Isabelle=False):
        """
        Generate a decision of whether to flip, using a percentage and then select randomly to flip
        """
        all_data = self.endog

        num_test_pairs = self.total_pairs
        flip_threshold = np.array((0,0.05,0.1,0.15,0.2,0.25,0.3,0.4,0.5))
        
        output_error_nihar = []
        output_error_isabelle = []

        for to_flip in flip_threshold:
            
            print 'Current evaluating with %f to flip' % to_flip

            test_pairs_with_error = self.pairs_truth

            num_to_flip = int(num_test_pairs*to_flip)

            for i in random.sample(range(num_test_pairs), num_to_flip):
                test_pairs_with_error[i] = (self.pairs_truth[i][1],self.pairs_truth[i][0])

            self.endog = test_pairs_with_error
            
            w_init = np.random.uniform(-5,5,video_num)
            self.fit(w_init)

            if showResult:
                res = self.rank[0]
                compare_rank(self.w_star, res, False, True)
                plt.legend([str(i) for i in num_test_pairs])  
            if Nihar:
                output_error_nihar.append(self.performance_nihar())
            if Isabelle:
                output_error_isabelle.append(self.performance_isabelle())
                
        self.endog = all_data
        return output_error_nihar, output_error_isabelle
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    
    def noise_generation(self, pair):
        """
        Generate a decision of whether to flip, using proability from normal distribution and then 
        Bernoulli to decide whether to flip
        """

        difference = self.rank[0][pair[0]] - self.rank[0][pair[1]]
        p = self.sigmoid(difference)*(1-self.sigmoid(difference))
        decision = bernoulli.rvs(p,size=1)
        
        return decision
    
    
    def test_data_generation_noise_generative_model(self, showResult=False, Nihar=False, Isabelle=False):
        """
        Function that calculate error using normal distribution to determine the probility of whether to
        flip the pair. Highest chance, at scores equal is around 0.12
        """
        all_data = self.endog

        test_pairs = self.pairs_truth
        
        for i in range(self.total_pairs):
            if self.noise_generation(test_pairs[i]):
                test_pairs[i][0] = test_pairs[i][1]
                test_pairs[i][1] = self.pairs_truth[i][0]
                
        self.endog = test_pairs
        w_init = np.random.uniform(-5,5,video_num)
        self.fit(w_init)

        if showResult:
            res = self.rank[0]
            compare_rank(self.w_star, res, False, True)
            plt.legend([str(i) for i in num_test_pairs])  
        if Nihar:
            print self.performance_nihar()
        if Isabelle:
            print self.performance_isabelle()

        self.endog = all_data
        
    def nloglikeobs(self, params):
        """
        Calculate negative log likelihood used for main optimization
        """
        
        out = 1
        pairs = self.endog
        w = params
        
        for pair in pairs:
            out *= 1/(1+np.exp(w[pair[1]] - w[pair[0]]))   
            
        return -np.log(out)
    
    
    def score(self, params):
        """
        Calculate gradient
        """
        self.rank.popleft()
        w = params
        pairs = self.endog
        grad = []
        for i in range(len(w)):
            grad.append(self.calc_gradient(pairs, w, i))
        
        self.rank.append(grad)
        
        return np.array(grad)
    
        
    def calc_gradient(self,pairs, w, w_i):
        """
        Function used in score() to calculate gradient
        """
        gradient = 0

        for pair in pairs:
            if w_i == pair[0]:
                out = -1
            elif w_i == pair[1]:
                out = 1  
            else:
                out = 0

            gradient -= out / (1/(np.exp(w[pair[1]]-w[pair[0]]) + 0.00001) +1 )
            
        return gradient
    
    
    def matching_func(self, param):
        """
        Function used to calculate L2 norm of v and w_star. Used to find a and b.
        Here params is [a, b]
        """
        return np.linalg.norm(self.w_star - param[0]*np.array(self.rank[0]) - param[1])

    
    def regularized_vector(self):
        """
        Function used to generate vector v after using matching_func to find optimal a and b
        """
        coeff = optimize.minimize(self.matching_func, [0, 0])
        
        self.a = coeff['x'][0]
        self.b = coeff['x'][1]
        self.v = self.a*np.array(self.rank[0])+self.b
        
        
    def performance_isabelle(self):
        """
        Performance evaluation using method proposed by Isabelle
        """
        epsilon = 0.0001
        error = 0
        self.regularized_vector()
        self.w_hat = self.v
    
        true_order = np.array(self.w_star).argsort()
        
        what = [ self.w_hat[i] for i in true_order]
        wstar = [ self.w_star[i] for i in true_order]
                 
        for i in range(len(self.w_star)-2):
            error += np.abs((wstar[i+1]-wstar[i])/(wstar[i+2]-wstar[i]+epsilon)-
                            (what[i+1]-what[i])/(what[i+2]-what[i]+epsilon))
        error /= self.total_pairs-2
        return error
    

    def performance_nihar(self):
        """
        Performance evaluation using method proposed by Nihar
        """
        self.regularized_vector()
        return np.linalg.norm(self.w_star - self.v)
    
    
    
    def fit(self, start_params=None, maxiter=10000, maxfun=50000):  
        return super(Pairwise, self).fit(start_params=start_params, 
                                         method='newton',
                                         bounds = (-5,5),
                                         maxiter=maxiter,
                                         disp=0,
                                         maxfun=maxfun)